# Working with HTTP

## What to Focus On

- **Focus on HTTP**: The focus of this lesson is still very much HTTP, and how it enables network communication between a client and a server.
- **HTTP is concerned with the structure of messages**: HTTP is a set of rules concerned with the syntax and structure of messages exchanged between applications. 
    - Working with HTTP is ultimately about understanding what those rules mean, and knowing how and when to apply them.
- **HTTP is a Request-Response protocol**: One of the fundamental aspects of HTTP is its Request-Response behavior.

## Using Telnet to explore HTTP

- HTTP is a text based protocol and underneath everything that happens is just text moving between the client and server.
- `nc -v google.com 80`: connect to `google.com` server on port `80`
- `GET /`: request the homepage
- `HTTP/1.0 200 OK`: response code and status.  `HTTP 1.0` is the version of http that the server is using.
- The information below are headers; a set of metadata that contains additional information about the response that doesn't actually go in the response body itself.
    - The `Content-Type` header is required most of the time and tells the browser or whatever the client is how to interpret the body of the response.  A browser will display the response visually.
    - A response can have any number of headers and that the headers themselves can contain a wide variety of different values based on what header it is and based on the information that is being passed for the value of the header.
    - The body contains HTML code that a browser would use to display the search page on `google.com`
```
Date: Tue, 28 Mar 2023 21:30:12 GMT
Expires: -1
Cache-Control: private, max-age=0
Content-Type: text/html; charset=ISO-8859-1
Content-Security-Policy-Report-Only: object-src 'none';base-uri 'self';script-src 'nonce-C-h8BMm--o_4NGmBnBncEQ' 'strict-dynamic' 'report-sample' 'unsafe-eval' 'unsafe-inline' https: http:;report-uri https://csp.withgoogle.com/csp/gws/other-hp
P3P: CP="This is not a P3P policy! See g.co/p3phelp for more info."
Server: gws
X-XSS-Protection: 0
X-Frame-Options: SAMEORIGIN
Set-Cookie: 1P_JAR=2023-03-28-21; expires=Thu, 27-Apr-2023 21:30:12 GMT; path=/; domain=.google.com; Secure
Set-Cookie: AEC=AUEFqZfN45gxu22DoWloHXGnWcVqG0rz-cGgzn9y1g748xm6fTd5EZd7opQ; expires=Sun, 24-Sep-2023 21:30:12 GMT; path=/; domain=.google.com; Secure; HttpOnly; SameSite=lax
Set-Cookie: NID=511=MqJGARbrSSfbYUusreymYAkAf0U5tAf5F9dgYbHjupU7j0oQ5Lg-xec3AxHVBwEbUuJTnnngoPWcon8kB0FElAu4kubx-GtuYdWHy4iAcHkFFDXrZElFgXGK2oybbZai0IIFz_SI1euX2QoKYc15ZFv3qVQ9fdpMFrUuNoNqVgs; expires=Wed, 27-Sep-2023 21:30:12 GMT; path=/; domain=.google.com; HttpOnly
Accept-Ranges: none
Vary: Accept-Encoding
```

## Speaking the Same Language

- In the video from the previous assignment, the request that we sent to `google.com` was very simple: `GET /`
- The server at `google.com` was able to interpret this as an HTTP request because the request followed certain syntactical rules, and the server was familiar with those rules. 
- The client making the request and the server receiving the request can speak the same language, and so are able to communicate.<br><br>
- There have been a number of different versions of HTTP since its inception, and each of those versions have slightly different rules.
- The structure of the request line we used for our request `GET /` follows the syntactical rules for HTTP 0.9 which only required the method and path in the request line.
- The response itself is formatted according to HTTP 1.0 syntax since it includes headers.<br><br>
- When making a request to `launchschool.com`, be sure to specify the HTTP version or the server may not respond: `GET / HTTP/1.1`.  This is because the web server does not support unspecified HTTP versions.
- The reponse looks like this:
```
HTTP/1.1 400 Bad Request
Cache-Control: no-cache, no-store
Content-Type: text/html; charset=utf-8
Date: 2023-03-28 21:44:52.514243727 +0000 UTC
Server: heroku-router
Content-Length: 0
```
- 4xx level response codes indicate an error or issue on the client side, i.e. with the request. 
- A `400` error is a general indication that there is a problem with the structure of the request; in other words the server did not understand the request due to its syntax. 
- This is basically the server saying to the client 'I don't understand what you just asked me'.<br><br>
- In versions of HTTP subsequent to 0.9, the request line must include the HTTP version, and so the server at `launchschool.com` is expecting this to be included in the request: `GET / HTTP/0.9`
- The response will look like this:
```
HTTP/1.1 505 HTTP Version Not Supported
Connection: close
Server: Cowboy
Date: Thu, 04 Jul 2019 11:13:59 GMT
Content-Length: 0
```
- 5xx level response codes indicate an error or issue on server side. This time the request was valid, and the server was able to understand it, but it replies to the client that it cannot communicate using the version of HTTP that the client wants to use.<br><br>
- Using a different HTTP version that the server should support: `GET / HTTP/1.1`
- We still get a `400` response.
- The specification for HTTP 1.1 states: `A client MUST send a Host header field in all HTTP/1.1 request messages.`
- Because we have specified that we want to use HTTP 1.1, the server is expecting a `Host` header in the request:
```
GET / HTTP/1.1
Host: launchschool.com

```
- We get a `301 Move Permanently` response code.
- Unlike 4xx and 5xx response codes, 3xx codes don't indicate an error. Instead they are generally used in relation to redirection, and indicate to the client that it must take some additional action in order to complete the request. 
- In this case, the server is set up to redirect all `http` requests to `https`. 
- The response includes a `Location` header which tells the client where it can now find the resource it originally requested. 
- If the client were a browser, rather than a terminal window using `telnet`, it would automatically issue a new request to the location indicated in the Location header. 
-  Rather than closing the TCP connection the server sets it to `keep-alive`; it does this since it is expecting another HTTP request from the client as a result of the `301` response.

## Implementing Our Own HTTP Server: Project Overview

- HTTP is essentially a set of rules for structuring messages between two participants in a message exchange. 
- It's up to the participants, for example a web browser and web server, to implement those rules. 
-  Ultimately it is the way that the rules are implemented which determines whether or not a message can be interpreted and also how messages are interpreted. 

## Implementing Our Own HTTP Server: Basic Program Structure

- The intention of our HTTP server is to conduct some processing of messages received from the client, and, based on certain rules that we define within the logic of the HTTP server program, issue an appropriate response.
-  Create a directory named `simple_http_server`. Within that directory create a file named `http_server.sh` and paste in the code provided.
```
#!/bin/bash

function server () {

}

coproc SERVER_PROCESS { server; }

netcat -lv 2345 <&${SERVER_PROCESS[0]} >&${SERVER_PROCESS[1]}
```
- On lines 3-5, we are defining a function called `server`. The body of the function is currently empty, this is where we will add the processing logic for our HTTP server program.
- On line 7 we are executing the `coproc` command. The coproc command is used to set up a coprocess: it allows us to run our `server` function asynchronously alongside our `netcat` command. 
- We are naming this process SERVER_PROCESS and telling it to execute the server function.
- On line 9 we are executing the netcat command.
    -  The first part of this line `netcat -lv 2345` executes `netcat` in listen and verbose mode, and sets it to listen for incoming connections on port 2345.
    - The second part allows for any input that `netcat` receives (for example a request message from a client) to be accessed within the `server` function using the `read` command, and anything that the server function outputs using `echo` will be output by `netcat`.

## Implementing Our Own HTTP Server: Sending a Simple Responses

- The aim of this assignment is for our server to be able to receive some input via `netcat` and then use that input as a part of a response. Use the following steps to implement this functionality:
    - Create a loop within server that will run indefinitely
    - Within the loop, assign any input received to a variable named message
    - Also within the loop, output the string You said: followed by the value of message.
```
#!/bin/bash

function server () {
  while true
  do
    read message
    echo "You said: $message"
  done
}

coproc SERVER_PROCESS { server; }

netcat -lvp 2345 <&${SERVER_PROCESS[0]} >&${SERVER_PROCESS[1]}
```
- You can test your solution by executing http_server.sh in one terminal window, connecting to the server in separate 'client' window, `netcat -v localhost 2345`, and sending some messages.

## Implementing Our Own HTTP Server: Processing the Request

- We've added some structure to our client-server exchange by sending a response for every message received. 
- We're still not really doing any meaningful processing of the received message on the server side, or imposing any kind of syntactic rules about how the message has to be structured.<br><br>
- We want to impose the following rules on any messages received:
    1. The messages, or requests, should consist of one line comprised of three parts: request method, path, HTTP version.
    2. The method should be GET since our server only responds to GET requests.
- Use the following steps to implement this functionality:
    1. Remove the current echo line
    2. Amend the read line to read the input to three variables: method, path, version.
    3. Below the read line create an if statement. It should test whether the method variable is equal to GET (we're not too interested in the path or the version at this stage).
    4. If the method is equal to GET then output the string HTTP/1.1 200 OK
    5. Add an else branch to the if statement that outputs the string HTTP/1.1 400 Bad Request

## Implementing Our Own HTTP Server: Serving HTML

- In the previous assignment we imposed some structure on any incoming requests and provided a meaningful response. 
- We're going to add a body to our response, and the content of that body will be taken from the HTML file that the request is for.

## Implementing Our Own HTTP Server: Working with the Browser

- Currently, our 'client' application is a terminal window running `netcat/ nc`.
- The requests it sends are one line messages, and it simply outputs the raw response it receives from the server to the screen. 
- In this context, the processing that we are doing on the server side is sufficient, and there is no processing at all occurring on the client side.